In [44]:
import json
import boto3
from typing import Dict, List
endpoint_name = "sausgenaipoc-dft-hf-llm-mistral-7b-ins-20240228-140547"


def query_endpoint(payload):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, InferenceComponentName='huggingface-llm-mistral-7b-instruct-20240228-140549', ContentType="application/json", Body=json.dumps(payload)
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response


def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for user, answer in zip(instructions[::2], instructions[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])
    prompt.extend(["<s>", "[INST] ", (instructions[-1]["content"]).strip(), " [/INST] "])
    return "".join(prompt)


def print_instructions(prompt: str, response: str) -> None:
    bold, unbold = '\033[1m', '\033[0m'
    print(f"{bold}> Input{unbold}\n{prompt}\n\n{bold}> Output{unbold}\n{response[0]['generated_text']}\n")
    
    


In [58]:
instructions = [{"role": "user", "content": "what is the recipe of mayonnaise?"}]

prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True}
}
response = query_endpoint(payload)
print_instructions(prompt, response)


> Input
<s>[INST] what is the recipe of mayonnaise? [/INST] 

> Output
1. Gather all of your ingredients:
- 2 eggs
- 1 tablespoon of dijon or white wine vinegar
- 1 cup of mayonnaise
- 1 tablespoon of fresh water
- Salt and pepper to taste

2. In a mixing bowl, whisk together the eggs, dijon or white wine vinegar, and 2 tablespoons of fresh water until the mixture is smooth and creamy.

3. Add in the mayonnaise and whisk until well combined.

4. Taste the mixture and add more salt and pepper to taste. If you find the mixture too thick, you can add more fresh water to thin it out.

5. Chill the mixture for at least 15 minutes before using it. This will allow the flavors to meld and the mixture to thicken slightly.

6. After chilling, give the mixture another whisk to make sure everything is smooth.

Your homemade mayonnaise is now ready to use! You can substitute the mayonnaise in any recipe that calls for it.



In [56]:
instructions = [
    {"role": "user", "content": "I am going to Paris, what should I see?"},
    {
        "role": "assistant",
        "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
    },
    {"role": "user", "content": "What is so great about #1?"},
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True}
}
response = query_endpoint(payload)
print_instructions(prompt, response)


> Input
<s>[INST] I am going to Paris, what should I see? [/INST] Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.</s><s>[INST] What is so great about #1? [/INST] 

> Output
1. The Eiffel Tower 

In [47]:
instructions = [
    {
        "role": "user",
        "content": "In Bash, how do I list all text files in the current directory (excluding subdirectories) that have been modified in the last month?",
    }
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True, "temperature": 0.2}
}
response = query_endpoint(payload)
print_instructions(prompt, response)


> Input
<s>[INST] In Bash, how do I list all text files in the current directory (excluding subdirectories) that have been modified in the last month? [/INST] 

> Output
1. Use the `find` command with the `-type f` option to search for files only, and the `-mtime -30` option to search for files modified in the last 30 days (1 month).
2. Use the `-name` option to search for files with a `.txt` extension.
3. Use the `-print` option to print the file names.

The command would look like this:

```bash
find . -type f -mtime -30 -name "*.txt" -print
```



In [57]:
instructions = [
    {
        "role": "user",
        "content": "What is the difference between inorder and preorder traversal? Give an example in Python.",
    }
]
prompt = format_instructions(instructions)
payload = {
    "inputs": prompt,
    "parameters": {"max_new_tokens": 256, "do_sample": True, "temperature": 0.2}
}
response = query_endpoint(payload)
print_instructions(prompt, response)

> Input
<s>[INST] What is the difference between inorder and preorder traversal? Give an example in Python. [/INST] 

> Output
1. Inorder traversal: Inorder traversal is a way of traversing a binary tree where we first traverse the left subtree, then the root node, and finally the right subtree. This is done in a left-root-right manner.

Example in Python:

```python
def inorder_traversal(root):
    if root:
        inorder_traversal(root.left)
        print(root.val, end=" ")
        inorder_traversal(root.right)
```

2. Preorder traversal: Preorder traversal is a way of traversing a binary tree where we first traverse the root node, then the left subtree, and finally the right subtree. This is done in a root-left-right manner.

Example in Python:

```python
def preorder_traversal(root):
    if root:
        print(root.val, end=" ")
        preorder_traversal(root.left)
        preorder_traversal(root.right)
```

In summary, the main difference between inorder and



In [33]:
instructions = [
    {
        "role": "user",
        "content": "Who is Shah Rukh Khan?",
    }
]
prompt = format_instructions(instructions)
print("printing format====>")
print(prompt)
payload = {
    "inputs": prompt,
    "parameters": {"do_sample": True, "temperature": 0.2}
}
print(payload)
response = query_endpoint(payload)
print_instructions(prompt, response)

printing format====>
<s>[INST] Who is Shah Rukh Khan? [/INST] 
{'inputs': '<s>[INST] Who is Shah Rukh Khan? [/INST] ', 'parameters': {'do_sample': True, 'temperature': 0.2}}
> Input
<s>[INST] Who is Shah Rukh Khan? [/INST] 

> Output
1.4 billion people know him as the Bollywood king, Shah Rukh Khan.



In [54]:
instructions = [{"role": "user", "content": "who is shah rukh khan?"}]

prompt = format_instructions(instructions)
payload = {
    "inputs": "who is shah rukh khan?",
    "parameters": {"max_new_tokens": 256, "do_sample": True}
}
print("=====request====")
print(payload)
response = query_endpoint(payload)
print("=====response====")
print(response)

=====request====
{'inputs': 'who is shah rukh khan?', 'parameters': {'max_new_tokens': 256, 'do_sample': True}}
=====response====
[{'generated_text': '\nA: Shah Rukh Khan is a Bollywood actor and producer of Indian films, who was born on November 2, 1965 in India. He is widely regarded as one of the greatest Bollywood actors of all time, having won numerous awards for his performances, including the National Film Award for the Best Actor, the Padma Shri, and the Filmfare Life Achievement Award. Khan is also the owner of the media company Red Chillies Entertainment, which includes the production company Red Chillies Films, and a studio in Mumbai, India.'}]
